In [11]:
from __future__ import annotations

from typing import Literal

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.axes import Axes
from matplotlib.figure import Figure

from src.drawing_utils import Point, Segment
from src.plotting_utils import configure_matplotlib, rm
from src.supply_demand import (
    DX,
    Colors,
    Curve,
    DemandCurve,
    Labels,
    SupplyCurve,
    SupplyDemand,
)
from src.supply_demand_plotter import CostUtilityPlotter, SupplyDemandPlotter

configure_matplotlib()

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(6.4, 4.2), layout="tight")

x_vals = np.linspace(0, 10)
supply_demand = SupplyDemand(
    [SupplyCurve([Point(x, 1 + 1 * x) for x in x_vals], stepped=False)],
    [DemandCurve([Point(x, 10 - x) for x in x_vals], stepped=False)],
    equilibrium_price=5.5,
)
SupplyDemandPlotter(ax1).plot_all(supply_demand)
Segment(supply_demand.equilibrium, Point(5.5, 5.5)).drawn(ax1).end.labeled(
    ax1, "$(Q^*, P^*)$", ha="left", va="center"
)
CostUtilityPlotter(ax2, ylim=(0, 60)).plot_all(supply_demand)

fig.savefig("img/fig_2_1.png", dpi=200)

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(6.4, 4.2), layout="tight")

supply_demand = SupplyDemand(
    [SupplyCurve([Point(0, 0), Point(6, 2), Point(9, 7)])],
    [DemandCurve([Point(0, 8), Point(4, 8), Point(8, 5)])],
    equilibrium_price=5,
)
SupplyDemandPlotter(ax1).plot_all(supply_demand)
Segment(supply_demand.equilibrium, (6.5, 4)).drawn(ax1).end.labeled(
    ax1, "$(Q^*, P^*)$", ha="left", va="top"
)

CostUtilityPlotter(ax2, ylim=(0, 50)).plot_all(supply_demand)

fig.savefig("img/fig_2_2.png", dpi=200)

In [4]:
cost_name = r"{name} Cost, $C_\mathrm{{name}}$"
utility_name = r"{name} Utility, $U_\mathrm{{name}}$"
dashed = "--"
dotted = ":"
supply_demand = SupplyDemand(
    supply_curves=[
        SupplyCurve(
            [Point(0, 0), Point(6, 2), Point(9, 7)], "G1", cost_name, fmt=dashed
        ),
        SupplyCurve(
            [Point(0, 0), Point(7, 4), Point(10, 10)], "G2", cost_name, fmt=dotted
        ),
    ],
    demand_curves=[
        DemandCurve(
            [Point(0, 8), Point(4, 8), Point(8, 5)], "L1", utility_name, fmt=dashed
        ),
        DemandCurve(
            [Point(0, 9), Point(3, 9), Point(9, 3)], "L2", utility_name, fmt=dotted
        ),
    ],
    equilibrium_price=4,
)


def two_generators_two_loads(style: Literal["ticks", "area"]) -> Figure:
    fig, axs = plt.subplots(nrows=2, ncols=2, layout="tight")

    for ax, curve in zip(np.ndarray.flatten(axs), supply_demand.curves, strict=True):
        equilibrium_quantity = (
            supply_demand.equilibrium_quantity(curve.name) if style == "area" else None
        )
        plotter = SupplyDemandPlotter(
            ax,
            xticks=(
                {equilibrium_quantity: rf"$Q_\mathrm{{{curve.name}}}^*$"}
                if style == "area"
                else None
            ),
            yticks=({} if style == "area" else None),
        )
        plotter.plot(curve, equilibrium_quantity)
        ax.set_title(rm(curve.name))

    return fig, axs

In [ ]:
fig, _ = two_generators_two_loads(style="ticks")
fig.savefig("img/fig_2_3.png", dpi=200)

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = two_generators_two_loads(style="area")

Point(3, 1).labeled(ax1, r"$C_\mathrm{G1}(Q_\mathrm{G1}^*)$")
Point(2.5, 2).labeled(ax2, r"$C_\mathrm{G2}(Q_\mathrm{G2}^*)$")
Point(4, 3.25).labeled(ax3, r"$U_\mathrm{L2}(Q_\mathrm{L2}^*)$")
Point(1.5, 4.5).labeled(ax4, r"$U_\mathrm{L2}(Q_\mathrm{L2}^*)$")

fig.savefig("img/fig_2_4.png", dpi=200)

In [7]:
def add_horizontal_brace(
    ax: Axes, x1: float, x2: float, y: float, label: str, opening: Literal["up", "down"]
) -> None:
    center_x = (x2 + x1) / 2
    sign = {"up": -1, "down": 1}[opening]
    ax.annotate(
        label,
        xy=(center_x, y + 0.5 * sign),
        xytext=(center_x, y + 1.5 * sign),
        ha="center",
        va={"up": "top", "down": "bottom"}[opening],
        arrowprops=dict(arrowstyle=f"-[, widthB={(x2 - x1) / 2 * 1.06 - 0.2}"),
    )

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(
    nrows=4,
    figsize=(6.4, 6.4),
    layout="tight",
    gridspec_kw=dict(height_ratios=[1 / 3, 1 / 6, 1 / 6, 1 / 6]),
)
xlim = (0, 20)

SupplyDemandPlotter(ax1, xlim).plot_all(supply_demand)
add_horizontal_brace(ax1, x1=0, x2=6, y=2, label=r"$Q_\mathrm{G1}^*$", opening="down")
add_horizontal_brace(ax1, x1=6, x2=11, y=4, label=r"$Q_\mathrm{G2}^*$", opening="up")
add_horizontal_brace(ax1, x1=0, x2=3, y=9, label=r"$Q_\mathrm{L1}^*$", opening="down")
add_horizontal_brace(ax1, x1=3, x2=11, y=8, label=r"$Q_\mathrm{L2}^*$", opening="down")
Segment(supply_demand.equilibrium, Point(14, 5)).drawn(ax1).end.labeled(
    ax1, "$(Q^*, P^*)$", ha="left"
)

ylim = (0, 100)
CostUtilityPlotter(ax2, xlim, ylim).plot_multiple(
    supply_demand.supply_curves, total=True
)
CostUtilityPlotter(ax3, xlim, ylim).plot_multiple(
    supply_demand.demand_curves, total=True
)
CostUtilityPlotter(ax4, xlim, ylim).plot_welfare(supply_demand)

fig.savefig("img/fig_2_5.png", dpi=200)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, figsize=(6.4, 6.4), layout="tight")
xlim = (0, 20)
quantity_vals = np.arange(xlim[0], xlim[1] + DX, DX)

loss_factor = 0.75


def plot_cost_and_utility(ax: Axes, key: Literal["G", "L"]) -> None:
    cost_vals = {
        "G": supply_demand.cost()(quantity_vals),
        "L": supply_demand.cost()(quantity_vals / loss_factor),
    }[key]
    utility_vals = {
        "G": supply_demand.utility()(quantity_vals * loss_factor),
        "L": supply_demand.utility()(quantity_vals),
    }[key]
    welfare_vals = utility_vals - cost_vals
    idxmax = np.nanargmax(welfare_vals)
    equilibrium_quantity = quantity_vals[idxmax]
    print(
        f"Q_opt = {equilibrium_quantity:.3f}, "
        f"C(Q_opt) = {cost_vals[idxmax]:.3f}, "
        f"U(Q_opt) = {utility_vals[idxmax]:.3f}, "
        f"W(Q_opt) = {welfare_vals[idxmax]:.3f}"
    )

    label = rf"Q_\mathrm{{{key}}}"
    plotter = SupplyDemandPlotter(
        ax,
        xlim,
        ylim=(0, 100),
        xticks={equilibrium_quantity: rf"${label}^*$"},
        yticks={},
        xaxis_label=rf"${label}$",
    )
    cost_or_utility = {
        "G": supply_demand.cost,
        "L": supply_demand.utility,
    }[key]
    for i in [1, 2]:
        [curve] = [c for c in supply_demand.curves if c.name == f"{key}{i}"]
        ax.plot(
            quantity_vals,
            cost_or_utility(curve.name)(quantity_vals),
            curve.fmt,
            color=curve.color,
            label=rm(curve.integral_name),
        )
    ax.plot(
        quantity_vals,
        cost_vals,
        SupplyCurve.fmt,
        color=SupplyCurve.color,
        label=rm(f"Total {Labels.COST}"),
    )
    ax.plot(
        quantity_vals,
        utility_vals,
        DemandCurve.fmt,
        color=DemandCurve.color,
        label=rm(f"Total {Labels.UTILITY}"),
    )
    ax.plot(quantity_vals, welfare_vals, color=Colors.WELFARE, label=rm(Labels.WELFARE))
    optimum = Point(equilibrium_quantity, np.nanmax(welfare_vals))
    ax.plot(*optimum.xy, "ko", markersize=3, label=rm(Labels.OPTIMUM))
    plotter.legend()

    return equilibrium_quantity


generator_equilibrium_quantity = plot_cost_and_utility(ax2, "G")
load_equilibrium_quantity = plot_cost_and_utility(ax3, "L")

plotter = SupplyDemandPlotter(ax1, xlim, xticks={}, yticks={})
plotter.plot(
    Curve.composite(supply_demand.supply_curves),
    equilibrium_quantity=generator_equilibrium_quantity,
)
plotter.plot(
    Curve.composite(supply_demand.demand_curves),
    equilibrium_quantity=load_equilibrium_quantity,
)
plotter.legend()
Segment(Point(generator_equilibrium_quantity, 4).drawn(ax1), Point(10.5, 7)).drawn(
    ax1
).end.labeled(ax1, r"$(Q_\mathrm{G}^*, P_\mathrm{G}^*)$", va="bottom")
Segment(Point(load_equilibrium_quantity, 8).drawn(ax1), Point(8, 10)).drawn(
    ax1
).end.labeled(ax1, r"$(Q_\mathrm{L}^*, P_\mathrm{L}^*)$", va="bottom")

fig.savefig("img/fig_2_6.png", dpi=200)